## Step1. 載入訓練資料

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random

from torch.utils.data import Dataset, DataLoader

TRAIN_NORMAL = "train_normal.npy"
TRAIN_INNER_BREAK = "train_inner_break.npy"
TRAIN_OUTER_BREAK = "train_outer_break.npy"
TEST_PREDICT = "test.npy"
TARGET_WIDTH = 32
TARGET_HEIGHT = 32
TARGET_LENGTH = TARGET_WIDTH * TARGET_HEIGHT

def random_crop(wave):
    start = np.random.randint(len(wave) - TARGET_LENGTH, size=1)[0]
    end = start + TARGET_LENGTH
    subwave = wave[start:end]
    return subwave

def type_to_lable(val):
    if "normal" in val:
        return 0
    if "inner_break" in val:
        return 1
    if "outer_break" in val:
        return 2
    raise Exception("unknown type")

def label_to_type(label):
    return ["normal", "inner_break", "outer_break", "unknown"][label]

def draw_wave(wave, label):
    y = wave
    x = np.arange(0, len(y), 1, dtype=int)
    plt.plot(x, y)
    plt.title(label_to_type(label))
    plt.show()

class WaveDatasets(Dataset):
    def __init__(self, loading=True):
        self._data_pair = list()

        if loading:
            for name in [TRAIN_NORMAL, TRAIN_INNER_BREAK, TRAIN_OUTER_BREAK]:
                data = np.load(name)
                label = type_to_lable(name)
                for wave in data:
                    self._data_pair.append((wave, label))
            random.shuffle(self._data_pair)

    def split(self, r=0.1):
        wset = WaveDatasets(False)
        size = round(len(self._data_pair) * (1. - 0.1))
        random.shuffle(self._data_pair)

        lhs = self._data_pair[:size]
        rhs = self._data_pair[size:]
        self._data_pair = lhs
        wset._data_pair = rhs
        return wset

    def look(self, idx):
        wave, label = self._data_pair[idx]
        draw_wave(wave, label)

    def __len__(self):
        return len(self._data_pair)

    def __getitem__(self, idx):
        wave, label = self._data_pair[idx]
        return torch.from_numpy(random_crop(wave)).float(), int(label)

## Step2. 架構網路

In [2]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class FullyConnect(nn.Module):
    def __init__(self, in_size,
                       out_size,
                       activation=None):
        super().__init__()
        self.act = activation
        self.linear = nn.Linear(
            in_size,
            out_size,
            bias=True
        )

    def forward(self, x):
        x = self.linear(x)
        if not self.act is None:
            x = self.act(x)
        return x

class ConvBlock(nn.Module):
    def __init__(self, in_channels,
                       out_channels,
                       kernel_size,
                       activation=None):
        super().__init__()
        self.act = activation
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            padding="same",
            bias=False,
        )
        self.bn = nn.BatchNorm2d(
            out_channels,
            eps=1e-5
        )
        nn.init.kaiming_normal_(self.conv.weight,
                                mode="fan_out",
                                nonlinearity="relu")
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        if not self.act is None:
            x = self.act(x)
        return x

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        oc = 1
        self._body = nn.Sequential(
            ConvBlock(1, 32, 3, nn.SiLU()),
            ConvBlock(32, 32, 3, nn.SiLU()),
            ConvBlock(32, 1, 3, nn.SiLU())
        )
        self._head = nn.Sequential(
            nn.Flatten(start_dim=1, end_dim=3),
            FullyConnect(oc * TARGET_WIDTH * TARGET_HEIGHT, 256, nn.SiLU()),
            FullyConnect(256, 3)
        )
        self._softmax = nn.Softmax(dim=1)

    def forward(self, x):
        b, n = x.shape
        x = torch.reshape(x, (b, 1, TARGET_WIDTH, TARGET_HEIGHT))
        x = self._body(x)
        x = self._head(x)
        return x

    def get_max(self, x):
        y = self.get_prob(x)
        _, i = torch.max(y, 1)
        return i.detach()

    def get_prob(self, x):
        y = self(x)
        y = self._softmax(y)
        return y.detach()

## Step3. 訓練網路

In [3]:
t_set = WaveDatasets()
v_set = t_set.split(r=0.1)

t_loader = DataLoader(t_set, batch_size=128, shuffle=True)
v_loader = DataLoader(v_set, batch_size=128, shuffle=True)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
net = Network()
net = net.to(device)

cross_entry = nn.CrossEntropyLoss()
opt = optim.Adam(net.parameters(),
                 lr=0.01,
                 weight_decay=0.)

for e in range(500):
    verbose_steps = 0
    running_loss = 0
    for _, (waves, labels) in enumerate(t_loader):
        net.train()
        waves = waves.to(device)
        labels = labels.to(device)
        
        pred = net(waves)
        loss = cross_entry(pred, labels)
    
        opt.zero_grad()
        loss.backward()
        opt.step()
    
        running_loss += loss.item()
        verbose_steps += 1

    correct = 0
    total = 0
    for _, (waves, labels) in enumerate(v_loader):
        net.eval()
        waves = waves.to(device)
        labels = labels.to(device)
        pred = net.get_max(waves)

        correct += torch.where(labels==pred, 1, 0).sum().item()
        total += len(labels)
    if (e+1) % 10 == 0:
        print("epoches: {} -> loss: {:.6f}, acc: {:.6f}%".format(
                  e+1, running_loss/verbose_steps, 100 * correct/total))

epoches: 10 -> loss: 0.004180, acc: 100.000000%
epoches: 20 -> loss: 0.006735, acc: 98.666667%
epoches: 30 -> loss: 0.006087, acc: 100.000000%
epoches: 40 -> loss: 0.000126, acc: 100.000000%
epoches: 50 -> loss: 0.043705, acc: 100.000000%
epoches: 60 -> loss: 0.029624, acc: 98.666667%
epoches: 70 -> loss: 0.004715, acc: 100.000000%
epoches: 80 -> loss: 0.007465, acc: 100.000000%
epoches: 90 -> loss: 0.057607, acc: 100.000000%
epoches: 100 -> loss: 0.000709, acc: 98.666667%
epoches: 110 -> loss: 0.005713, acc: 100.000000%
epoches: 120 -> loss: 0.000349, acc: 100.000000%
epoches: 130 -> loss: 0.000132, acc: 100.000000%
epoches: 140 -> loss: 0.001699, acc: 100.000000%
epoches: 150 -> loss: 0.000007, acc: 100.000000%
epoches: 160 -> loss: 0.000021, acc: 100.000000%
epoches: 170 -> loss: 0.000011, acc: 100.000000%
epoches: 180 -> loss: 0.000168, acc: 100.000000%
epoches: 190 -> loss: 0.000277, acc: 100.000000%
epoches: 200 -> loss: 0.000147, acc: 100.000000%
epoches: 210 -> loss: 0.003283, 

## Step4. 預測節果並輸出

In [4]:
import pandas as pd
import io

csv_result = str()
net.eval()
data = np.load(TEST_PREDICT)

for idx, wave in enumerate(data):
    subwaves = list()
    for i in range(128):
        subwaves.append(random_crop(wave))
    subwaves = np.array(subwaves)
    subwaves = torch.from_numpy(subwaves).float()
    subwaves = subwaves.to(device)
    pred = net.get_max(subwaves)

    result = np.zeros((3))
    for i in range(len(result)):
        result[i] = torch.where(pred==i, 1, 0).sum().item()
    csv_result += "{}\n".format(label_to_type(np.argmax(result)))

csv_result = csv_result[:-1]
df = pd.read_csv(io.StringIO(csv_result), sep=",", header=None)
print(df)
with open("HW3.csv", "w") as f:
    f.write(csv_result)

               0
0    inner_break
1         normal
2    inner_break
3    outer_break
4    outer_break
..           ...
235  inner_break
236  inner_break
237       normal
238  inner_break
239       normal

[240 rows x 1 columns]
